In [ ]:
import sys
sys.path.append('/Users/vdk/Software/code/')
import h5py
import pandas as pd
from astropy.table import Table, vstack
import numpy as np
from scipy.interpolate import interp1d
from ctapipe.io import EventSource
import statsmodels.api as sm
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
import glob
#from python_modules.muonpipe import usefull_func
import seaborn as sns
from ctapipe.image import tailcuts_clean
import scipy.stats as stats
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import glob
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    'k'
]

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
filename = '/Users/vdk/muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
#source = EventSource(filename, max_events = 1)
#camgeom = source.subarray.tel[1].camera.geometry


### Base Cuts

In [ ]:
size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

### Data loading

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))

df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[11:]] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data_real = df_all_data[(df_all_data['muon_efficiency'] < muon_efficiency_cut) & (df_all_data['size_outside'] < size_outside_cut)]
del(df_all_data)
df_good_data_real

In [ ]:
ring_completeness_cut = 0.99
ring_containment_cut = 0.3

fits_file = '/Users/vdk/muons2024/data/20240310fits/muons_LST-1.Run17043.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]


event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

# This is required for further cuts in the simulations
min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = '/Users/vdk/muons2024/data/17043_muons_08complet.txt'
#np.save(filename_event_ids, event_ids)

df_real_data = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

df_real_data

### One night obs

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/data/20240310fits/*')

dat = Table.read('{}'.format(muon_files[0]), format='fits')

ring_completeness_cut = 0.3
ring_containment_cut = 0.3

for muon_file in muon_files:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_real_data_stacked = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500) &
                      (df['ring_completeness'] > ring_completeness_cut) &
                      (df['ring_containment'] > ring_containment_cut)]


df_real_data_stacked

### 1 sim loading

In [ ]:
global_cor_nsbtune_fits_file = '/Users/vdk/muons2024/config_comparison/integration_correction/triple_tuning/muon_table_run1_nsb_tune.fits'
global_cor_nsbtune_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction/triple_tuning/dl1_run101_muon.h5'

ring_completeness_cut = 0.8
ring_containment_cut = 0.99

dat = Table.read(global_cor_nsbtune_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < muon_efficiency_cut) & 
                  (df['size_outside'] < size_outside_cut) & 
                  (df['impact_parameter'] < max_impact) &
                  (df['impact_parameter'] > min_impact)
                 ]


event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor_nsbtune = df_good_data
                                        

global_cor_nsbtune_sim_dl1_image = []
global_cor_nsbtune_sim_dl1_peak_time = []
global_cor_nsbtune_sim_dl1_image_mask = []

with h5py.File(global_cor_nsbtune_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_cor_nsbtune_sim_dl1_image.append(event_record[2])
            global_cor_nsbtune_sim_dl1_peak_time.append(event_record[3])
            global_cor_nsbtune_sim_dl1_image_mask.append(event_record[4])
        #if len(global_cor_nsbtune_sim_dl1_image) == 542:
        #    break


df_sim_data_global_integration_cor_nsbtune

### List of fits sim files loading

#### NSB tuned to years

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/grand_tune/2020/tables/*')
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/grand_tune/2021/tables/*')
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/grand_tune/2022/tables/*')
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/grand_tune/2023/tables/*')
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/nsb_tune_2024_year_no_outliers/tables/*')

listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2020year_tuned_reflectivity/tables/*')
#listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2021year_tuned_reflectivity/tables/*')
#listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2023year_tuned_reflectivity/tables/*')
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_2024_tune = df[df['good_ring'] & (df['muon_efficiency'] < muon_efficiency_cut) & (df['size_outside'] < size_outside_cut)]
df_sim_2024_tune

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/no_tune_zenith_10/muon_tables/*')
dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_zenith10 = df[df['good_ring'] & (df['muon_efficiency'] < muon_efficiency_cut) & (df['size_outside'] < size_outside_cut)]
df_sim_zenith10

### Creating dataframes with cuts to plot

In [ ]:
ring_containment_cut = 0.3
ring_completeness_cut = 0.99
min_radius_cut = 0.95
max_radius_cut = 1.15


df_real_data_cut_2020 = df_good_data_real[ 
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &  
    (df_good_data_real['ring_radius'] < max_radius_cut) &  
    (df_good_data_real['event_time'] > start_date_2020) &
    (df_good_data_real['event_time'] < end_date_2020)
]

# df_real_data_cut = df_good_data_real[ 
#     (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#     (df_good_data_real['ring_containment'] > ring_containment_cut) &
#     (df_good_data_real['ring_radius'] > min_radius_cut) &  
#     (df_good_data_real['ring_radius'] < max_radius_cut) &  
#     (df_good_data_real['event_time'] > start_date_2022) &
#     (df_good_data_real['event_time'] < end_date_2022)
# ]

df_real_data_cut_2021 = df_good_data_real[ 
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &  
    (df_good_data_real['ring_radius'] < max_radius_cut) &  
    (df_good_data_real['event_time'] > start_date_2021) &
    (df_good_data_real['event_time'] < end_date_2021)
]

df_real_data_cut_2022 = df_good_data_real[ 
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &  
    (df_good_data_real['ring_radius'] < max_radius_cut) &  
    (df_good_data_real['event_time'] > start_date_2022) &
    (df_good_data_real['event_time'] < end_date_2022)
]

df_real_data_cut_2023 = df_good_data_real[ 
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &  
    (df_good_data_real['ring_radius'] < max_radius_cut) &  
    (df_good_data_real['event_time'] > start_date_2023) &
    (df_good_data_real['event_time'] < end_date_2023)
]

df_real_data_cut_2024 = df_good_data_real[ 
    (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
    (df_good_data_real['ring_containment'] > ring_containment_cut) &
    (df_good_data_real['ring_radius'] > min_radius_cut) &  
    (df_good_data_real['ring_radius'] < max_radius_cut) &  
    (df_good_data_real['event_time'] > start_date_2024) &
    (df_good_data_real['event_time'] < end_date_2024)
]

# df_sim_cut_nsbtune = df_sim_data_global_integration_cor_nsbtune[
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > min_radius_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < max_radius_cut) &
#     (df_sim_data_global_integration_cor_nsbtune['ring_completeness'] > ring_completeness_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_containment'] > ring_containment_cut) 
# ]

# df_sim_cut = df_sim_zenith10[
#     (df_sim_zenith10['ring_radius'] > min_radius_cut) & 
#     (df_sim_zenith10['ring_radius'] < max_radius_cut) &
#     (df_sim_zenith10['ring_completeness'] > ring_completeness_cut) & 
#     (df_sim_zenith10['ring_containment'] > ring_containment_cut) 
# ]

df_sim_cut_2024_tune = df_sim_2024_tune[
    (df_sim_2024_tune['ring_radius'] > min_radius_cut) & 
    (df_sim_2024_tune['ring_radius'] < max_radius_cut) &
    (df_sim_2024_tune['ring_completeness'] > ring_completeness_cut) & 
    (df_sim_2024_tune['ring_containment'] > ring_containment_cut) 
]





#print(f"Size of the sim dataset {len(df_sim_cut)}")
print(f"Size of the sim 2024 tune dataset {len(df_sim_cut_2024_tune)}")
#print(f"Size of the obs dataset {len(df_real_data_cut)}")



## Plotting of the regression data vs sim

In [ ]:
year_value = 2024
data_mc = df_sim_cut_2024_tune
# data_mc = df_sim_data_global_integration_cor_nsbtune[
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > min_radius_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < max_radius_cut) &
#     (df_sim_data_global_integration_cor_nsbtune['ring_completeness'] > ring_completeness_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_containment'] > ring_containment_cut) 
# ]
data_run = eval(f"df_real_data_cut_{year_value}")
#data_run = df_real_data_stacked

# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 10)
# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)


# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size'] * 0.998
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()


# # Bin the data for MC
# df_real_data_cut_2021['binned_radius'] = pd.cut(df_real_data_cut_2021['ring_radius'], bins, include_lowest=True, labels=False)
# binned_data_2021 = df_real_data_cut_2021.groupby('binned_radius').agg({
#     'ring_radius': 'mean',
#     'ring_size': 'mean'
# })
# binned_data_2021['ring_size'] = binned_data_2021['ring_size']
# binned_data_2021['constant'] = 1
# model_data_2021 = sm.OLS(binned_data_2021['ring_size'], binned_data_2021[['constant', 'ring_radius']]).fit()
# # Generate y values and confidence intervals using the regression models
# y_values_data_2021 = model_data_2021.predict(x_values_with_const)
# prediction_data_2021 = model_data_2021.get_prediction(x_values_with_const)
# conf_int_mc_2021 = prediction_data_2021.conf_int()


# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run['constant'] = 1
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()
y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()


# Plotting
plt.figure(figsize=(12, 12))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation with NSB and optical efficiency tuned to 2024 year ', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# plt.scatter(binned_data_2021['ring_radius'], binned_data_2021['ring_size'], color='blue', label='Simulation with tuned NSB to Run17043 (0.31 GHz)', s=50)
# plt.plot(x_values, y_values_data_2021, color='blue', linestyle='--', label='MC regression line')
# plt.fill_between(x_values, conf_int_data_2021[:, 0], conf_int_data_2021[:, 1], color='', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label=f'Data for {year_value} year', s=50)

plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)


# Adding labels and legend
plt.xlabel('Muon radius [deg]',fontsize=16)
plt.ylabel('Muon ring size [p.e.]',fontsize=16)
plt.legend()
plt.ylim(1800,3000)
#plt.grid(alpha=0.5)
#plt.title(f"Quality cuts containment > {ring_containment_cut} completeness > {ring_completeness_cut}")
#plt.show()
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/cuts_investigation/2022tune_size_vs_radius_{year_value}year_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')
#plt.savefig(f'/Users/vdk/muons2024/images/AA_paper/2024tune_size_vs_radius_{year_value}year_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
z,x,c = plt.hist(df_real_data_cut_2022['ring_radius'], bins=10, histtype='step', color='r', label='2022',density=True)
z,x,c = plt.hist(df_sim_cut_2024_tune['ring_radius'], bins=10, histtype='step', color='b', label='2022', density=True)

### Slope of the regressiobn line

In [ ]:
#x_values,y_values_run

In [ ]:
slope = model_mc.params['ring_radius']
# in p.e. per degree
slope

In [ ]:
np.degrees(np.arctan(slope/1000))

In [ ]:
print(f"2020 year: Size of the cutted dataset  {len(df_real_data_cut_2020)} vs whole dataset "
      f"{len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2020) & (df_good_data_real['event_time'] < end_date_2020)])} "
      f"which means {round(len(df_real_data_cut_2020)/len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2020) & (df_good_data_real['event_time'] < end_date_2020)])*100,3)} %")

print(f"2021 year: Size of the cutted dataset  {len(df_real_data_cut_2021)} vs whole dataset "
      f"{len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2021) & (df_good_data_real['event_time'] < end_date_2021)])} "
      f"which means {round(len(df_real_data_cut_2021)/len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2021) & (df_good_data_real['event_time'] < end_date_2021)])*100,3)} %")

print(f"2022 year: Size of the cutted dataset {len(df_real_data_cut_2022)} vs whole dataset "
      f"{len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2022) & (df_good_data_real['event_time'] < end_date_2022)])} "
      f"which means {round(len(df_real_data_cut_2022)/len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2022) & (df_good_data_real['event_time'] < end_date_2022)])*100,3)} %")
      
print(f"2023 year: Size of the cutted dataset {len(df_real_data_cut_2023)} vs whole dataset "
      f"{len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2023) & (df_good_data_real['event_time'] < end_date_2023)])} "
      f"which means {round(len(df_real_data_cut_2023)/len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2023) & (df_good_data_real['event_time'] < end_date_2023)])*100,3)} %")

print(f"2024 year: Size of the cutted dataset  {len(df_real_data_cut_2024)} vs whole dataset " 
      f"{len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2024) & (df_good_data_real['event_time'] < end_date_2024)])} "
      f"which means {round(len(df_real_data_cut_2024)/len(df_good_data_real[(df_good_data_real['event_time'] > start_date_2024) & (df_good_data_real['event_time'] < end_date_2024)])*100,3)} %")

#### For all years in one plot to trace the changes

In [ ]:

data_mc = df_sim_cut



# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 9)
# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)


# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size']
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()


# Bin the data for 2020 year
df_real_data_cut_2020['binned_radius'] = pd.cut(df_real_data_cut_2020['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2020 = df_real_data_cut_2020.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2020['constant'] = 1
model_run = sm.OLS(binned_run_2020['ring_size'], binned_run_2020[['constant', 'ring_radius']]).fit()
y_values_run_2020 = model_run.predict(x_values_with_const)
prediction_run_2020 = model_run.get_prediction(x_values_with_const)
conf_int_run_2020 = prediction_run_2020.conf_int()

# Bin the data for 2021 year
df_real_data_cut_2021['binned_radius'] = pd.cut(df_real_data_cut_2021['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2021 = df_real_data_cut_2021.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2021['constant'] = 1
model_run = sm.OLS(binned_run_2021['ring_size'], binned_run_2021[['constant', 'ring_radius']]).fit()
y_values_run_2021 = model_run.predict(x_values_with_const)
prediction_run_2021 = model_run.get_prediction(x_values_with_const)
conf_int_run_2021 = prediction_run_2021.conf_int()

# Bin the data for 2022 year
df_real_data_cut_2022['binned_radius'] = pd.cut(df_real_data_cut_2022['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2022 = df_real_data_cut_2022.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2022['constant'] = 1
model_run = sm.OLS(binned_run_2022['ring_size'], binned_run_2022[['constant', 'ring_radius']]).fit()
y_values_run_2022 = model_run.predict(x_values_with_const)
prediction_run_2022 = model_run.get_prediction(x_values_with_const)
conf_int_run_2022 = prediction_run_2022.conf_int()

# Bin the data for 2023 year
df_real_data_cut_2023['binned_radius'] = pd.cut(df_real_data_cut_2023['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2023 = df_real_data_cut_2023.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2023['constant'] = 1
model_run = sm.OLS(binned_run_2023['ring_size'], binned_run_2023[['constant', 'ring_radius']]).fit()
y_values_run_2023 = model_run.predict(x_values_with_const)
prediction_run_2023 = model_run.get_prediction(x_values_with_const)
conf_int_run_2023= prediction_run_2023.conf_int()

# Bin the data for 2024 year
df_real_data_cut_2024['binned_radius'] = pd.cut(df_real_data_cut_2024['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2024 = df_real_data_cut_2024.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2024['constant'] = 1
model_run = sm.OLS(binned_run_2024['ring_size'], binned_run_2024[['constant', 'ring_radius']]).fit()
y_values_run_2024 = model_run.predict(x_values_with_const)
prediction_run_2024 = model_run.get_prediction(x_values_with_const)
conf_int_run_2024 = prediction_run_2024.conf_int()


# Plotting
plt.figure(figsize=(12, 8))



# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)


plt.scatter(binned_run_2020['ring_radius'], binned_run_2020['ring_size'], color='red', label='Data from 2020 year', s=50, alpha=0.6, marker = '^')
plt.plot(x_values, y_values_run_2020, color='red', linestyle='--', alpha=0.8)
plt.fill_between(x_values, conf_int_run_2020[:, 0], conf_int_run_2020[:, 1], color='red', alpha=0.2)


# #plt.scatter(binned_run_2021['ring_radius'], binned_run_2021['ring_size'], color='green', label='Data from 20221year', s=50)
# plt.plot(x_values, y_values_run_2021, color='green', linestyle='--', alpha=0.8, label='Data from 2021 year')


# # Plotting Data Run
plt.scatter(binned_run_2022['ring_radius'], binned_run_2022['ring_size'], color='orange', label='Data from 2022 year', s=50, marker = 's')
plt.plot(x_values, y_values_run_2022, color='orange', linestyle='--')
plt.fill_between(x_values, conf_int_run_2022[:, 0], conf_int_run_2022[:, 1], color='orange', alpha=0.2)


#plt.scatter(binned_run_2023['ring_radius'], binned_run_2023['ring_size'], color='violet', label='Data from 2023 year', s=50, alpha=0.6)
# plt.plot(x_values, y_values_run_2023, color='violet', linestyle='--', alpha=0.8, label='Data from 2023 year')



plt.scatter(binned_run_2024['ring_radius'], binned_run_2024['ring_size'], color='green', label='Data from 2024 year', s=50, alpha=0.6, marker = 'v')
plt.plot(x_values, y_values_run_2024, color='green', linestyle='--', alpha=0.8)
plt.fill_between(x_values, conf_int_run_2024[:, 0], conf_int_run_2024[:, 1], color='green', alpha=0.2)

# Adding labels and legend
plt.xlabel('Muon radius [deg]',fontsize=16)
plt.ylabel('Muon ring size [p.e.]',fontsize=16)
plt.legend()
plt.ylim(2000,2800)
plt.grid(alpha=0.5)
#plt.title(f"Quality cuts containment = {ring_containment_cut} completeness = {ring_completeness_cut}")
#plt.show()
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/cuts_investigation/three_years_one_plot_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

### Normalization of the data to extract the correct slope

In [ ]:
data_mc = df_sim_cut

# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 9)
# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)


# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size']
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()


# Bin the data for 2020 year
df_real_data_cut_2020['binned_radius'] = pd.cut(df_real_data_cut_2020['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2020 = df_real_data_cut_2020.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2020['constant'] = 1
model_run = sm.OLS(binned_run_2020['ring_size'], binned_run_2020[['constant', 'ring_radius']]).fit()
y_values_run_2020 = model_run.predict(x_values_with_const)
prediction_run_2020 = model_run.get_prediction(x_values_with_const)
conf_int_run_2020 = prediction_run_2020.conf_int()

# Bin the data for 2021 year
df_real_data_cut_2021['binned_radius'] = pd.cut(df_real_data_cut_2021['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2021 = df_real_data_cut_2021.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2021['constant'] = 1
model_run = sm.OLS(binned_run_2021['ring_size'], binned_run_2021[['constant', 'ring_radius']]).fit()
y_values_run_2021 = model_run.predict(x_values_with_const)
prediction_run_2021 = model_run.get_prediction(x_values_with_const)
conf_int_run_2021 = prediction_run_2021.conf_int()

# Bin the data for 2022 year
df_real_data_cut_2022['binned_radius'] = pd.cut(df_real_data_cut_2022['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2022 = df_real_data_cut_2022.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2022['constant'] = 1
model_run = sm.OLS(binned_run_2022['ring_size'], binned_run_2022[['constant', 'ring_radius']]).fit()
y_values_run_2022 = model_run.predict(x_values_with_const)
prediction_run_2022 = model_run.get_prediction(x_values_with_const)
conf_int_run_2022 = prediction_run_2022.conf_int()

# Bin the data for 2023 year
df_real_data_cut_2023['binned_radius'] = pd.cut(df_real_data_cut_2023['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2023 = df_real_data_cut_2023.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2023['constant'] = 1
model_run = sm.OLS(binned_run_2023['ring_size'], binned_run_2023[['constant', 'ring_radius']]).fit()
y_values_run_2023 = model_run.predict(x_values_with_const)
prediction_run_2023 = model_run.get_prediction(x_values_with_const)
conf_int_run_2023= prediction_run_2023.conf_int()

# Bin the data for 2024 year
df_real_data_cut_2024['binned_radius'] = pd.cut(df_real_data_cut_2024['ring_radius'], bins, include_lowest=True, labels=False)
binned_run_2024 = df_real_data_cut_2024.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run_2024['constant'] = 1
model_run = sm.OLS(binned_run_2024['ring_size'], binned_run_2024[['constant', 'ring_radius']]).fit()
y_values_run_2024 = model_run.predict(x_values_with_const)
prediction_run_2024 = model_run.get_prediction(x_values_with_const)
conf_int_run_2024 = prediction_run_2024.conf_int()

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Function to normalize a column in a DataFrame
def normalize(column):
    return (column - column.mean()) / column.std()

# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 9)

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)

# Normalize x_values
x_values_norm = (x_values - np.mean(x_values)) / np.std(x_values)
x_values_with_const = sm.add_constant(x_values_norm)

# List of DataFrames for each year
dataframes = [df_real_data_cut_2020, df_real_data_cut_2021, df_real_data_cut_2022, df_real_data_cut_2023, df_real_data_cut_2024]
models = {}
predictions = {}
conf_ints = {}

# Process each year
for year, df in zip([2020, 2021, 2022, 2023, 2024], dataframes):
    df['binned_radius'] = pd.cut(df['ring_radius'], bins, include_lowest=True, labels=False)
    binned_df = df.groupby('binned_radius').agg({
        'ring_radius': 'mean',
        'ring_size': 'mean'
    })

    # Normalize the binned data
    binned_df['ring_radius'] = normalize(binned_df['ring_radius'])
    binned_df['ring_size'] = normalize(binned_df['ring_size'])
    binned_df['constant'] = 1

    # Fit the model
    model = sm.OLS(binned_df['ring_size'], binned_df[['constant', 'ring_radius']]).fit()
    models[year] = model

    # Generate predictions and confidence intervals
    y_values = model.predict(x_values_with_const)
    prediction = model.get_prediction(x_values_with_const)
    conf_int = prediction.conf_int()

    predictions[year] = y_values
    conf_ints[year] = conf_int

    # Print the slope of the regression line
    slope = model.params['ring_radius']
    angle = np.arctan(slope) * (180 / np.pi)
    print(f"Slope of the regression line for {year}: {slope}")
    print(f"Angle of the regression line for {year}: {angle} degrees")

# Example for MC data
data_mc = df_sim_cut
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Normalize the binned data
binned_mc['ring_radius'] = normalize(binned_mc['ring_radius'])
binned_mc['ring_size'] = normalize(binned_mc['ring_size'])
binned_mc['constant'] = 1

# Fit the model
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()

# Generate predictions and confidence intervals
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

# Print the slope of the regression line for MC data
slope_mc = model_mc.params['ring_radius']
angle_mc = np.arctan(slope_mc) * (180 / np.pi)
print(f"Slope of the regression line for MC data: {slope_mc}")
print(f"Angle of the regression line for MC data: {angle_mc} degrees")

# Store results for MC data
predictions['MC'] = y_values_mc
conf_ints['MC'] = conf_int_mc

# Now you can use the 'models', 'predictions', and 'conf_ints' dictionaries for further analysis or plotting

In [ ]:
np.mean(binned_mc['ring_size'])

In [ ]:
prediction_run_2024

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Assuming data_mc, df_real_data_cut_2020, df_real_data_cut_2021, df_real_data_cut_2022, df_real_data_cut_2023, df_real_data_cut_2024, min_radius_cut, max_radius_cut are already defined

data_mc = df_sim_cut
# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 9)
# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Function to fit OLS model and return intercept and other relevant values
def fit_ols_model(data, bins):
    data['binned_radius'] = pd.cut(data['ring_radius'], bins, include_lowest=True, labels=False)
    binned_data = data.groupby('binned_radius').agg({
        'ring_radius': 'mean',
        'ring_size': 'mean'
    })
    binned_data['constant'] = 1
    model = sm.OLS(binned_data['ring_size'], binned_data[['constant', 'ring_radius']]).fit()
    return model, binned_data

# Fit OLS models
model_mc, binned_mc = fit_ols_model(data_mc, bins)
model_2020, binned_2020 = fit_ols_model(df_real_data_cut_2020, bins)
model_2021, binned_2021 = fit_ols_model(df_real_data_cut_2021, bins)
model_2022, binned_2022 = fit_ols_model(df_real_data_cut_2022, bins)
model_2023, binned_2023 = fit_ols_model(df_real_data_cut_2023, bins)
model_2024, binned_2024 = fit_ols_model(df_real_data_cut_2024, bins)

# Generate y values and confidence intervals using the regression models
def get_model_predictions(model):
    y_values = model.predict(x_values_with_const)
    prediction = model.get_prediction(x_values_with_const)
    conf_int = prediction.conf_int()
    return y_values, conf_int

y_values_mc, conf_int_mc = get_model_predictions(model_mc)
y_values_run_2020, conf_int_run_2020 = get_model_predictions(model_2020)
y_values_run_2021, conf_int_run_2021 = get_model_predictions(model_2021)
y_values_run_2022, conf_int_run_2022 = get_model_predictions(model_2022)
y_values_run_2023, conf_int_run_2023 = get_model_predictions(model_2023)
y_values_run_2024, conf_int_run_2024 = get_model_predictions(model_2024)

# Calculate the difference between normalizations (intercepts) for 2022 and 2024
intercept_2020 = model_2020.params['constant']
intercept_2021 = model_2021.params['constant']
intercept_2022 = model_2022.params['constant']
intercept_2023 = model_2023.params['constant']
intercept_2024 = model_2024.params['constant']
intercept_mc = model_mc.params['constant']
difference_in_intercepts = intercept_2024 - intercept_2022

print(f"Normalization (intercept) for 2020: {intercept_2020}")
print(f"Normalization (intercept) for 2021: {intercept_2021}")
print(f"Normalization (intercept) for 2022: {intercept_2022}")
print(f"Normalization (intercept) for 2023: {intercept_2023}")
print(f"Normalization (intercept) for 2024: {intercept_2024}")
print(f"Normalization (intercept) for mc: {intercept_mc}")
print(f"Difference in normalization between 2022 and 2024: {difference_in_intercepts}")

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
#plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
#plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

plt.plot(x_values, y_values_run_2020, color='red', linestyle='--', alpha=0.8, label='Regression line for 2020 year')

plt.plot(x_values, y_values_run_2021, color='green', linestyle='--', alpha=0.8, label='Regression line for 2021 year')

#plt.scatter(binned_2022['ring_radius'], binned_2022['ring_size'], color='orange', label='Data from 2022 year', s=50)
plt.plot(x_values, y_values_run_2022, color='orange', linestyle='--', label='Regression line for 2022 year')
#plt.fill_between(x_values, conf_int_run_2022[:, 0], conf_int_run_2022[:, 1], color='orange', alpha=0.2)

plt.plot(x_values, y_values_run_2023, color='violet', linestyle='--', alpha=0.8, label='Regression line for 2023 year')
plt.plot(x_values, y_values_run_2024, color='darkturquoise', linestyle='--', alpha=0.8, label='Regression line for 2024 year')

# Adding labels and legend
plt.xlabel('Muon radius [deg]', fontsize=16)
plt.ylabel('Muon ring size [p.e.]', fontsize=16)
plt.legend()
plt.ylim(2000, 2800)
plt.grid(alpha=0.5)
plt.show()

#print(f"Difference in normalization (intercepts) between 2022 and 2024: {difference_in_intercepts}")
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/cuts_investigation/all_years_just_regression_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
#np.mean(binned_2022['ring_size'] - binned_2024['ring_size'])/np.mean(binned_2022['ring_size'])*100

In [ ]:
np.mean(y_values_run_2022 - y_values_run_2024)/np.mean(y_values_run_2022)

In [ ]:
(binned_2022['ring_size'] - binned_2024['ring_size']) / np.mean(binned_2022['ring_size'])

### 

###

In [ ]:
# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 8)

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# To try renormalization if needed
binned_mc['ring_size'] = binned_mc['ring_size']

# Perform linear regression
mc_slope, mc_intercept = np.polyfit(binned_mc['ring_radius'], binned_mc['ring_size'], 1)
run_slope, run_intercept = np.polyfit(binned_run['ring_radius'], binned_run['ring_size'], 1)

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)

# Generate y values using the regression parameters
y_values_mc = mc_slope * x_values + mc_intercept
y_values_run = run_slope * x_values + run_intercept

# Plotting
plt.figure(figsize=(10, 6))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Standard config', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression')

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='2024 data', s=50)
plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression')

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.grid(alpha=0.5)
plt.show()

In [ ]:
plt.hist(df_good_data_real['ring_containment'][df_good_data_real['ring_completeness'] > 0.99],density=True, bins = 10)
#plt.hist(df_good_data_real['ring_containment'],density=True, bins = 10, alpha = 0.7)
#plt.ylim(0,2)
plt.xlabel('Ring containment')
plt.ylabel('Probability density')
plt.xlim(0,1.1)
plt.grid(alpha = 0.3)
plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/lst_reco_call/containment_only_full_rings.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.hist(df_good_data_real['ring_completeness'],density=True, bins = 10, alpha = 0.7, histtype='step', linewidth = 2)
plt.hist(df_sim_['ring_completeness'],density=True, bins=10, histtype='step', linewidth = 2, label='Simulation')
plt.grid(alpha = 0.3)
plt.xlabel('Ring completeness')
plt.ylabel('Probability density')
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/lst_reco_call/completeness.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
year_value = 2022
data_mc = df_sim_cut
# data_mc = df_sim_data_global_integration_cor_nsbtune[
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > min_radius_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < max_radius_cut) &
#     (df_sim_data_global_integration_cor_nsbtune['ring_completeness'] > ring_completeness_cut) & 
#     (df_sim_data_global_integration_cor_nsbtune['ring_containment'] > ring_containment_cut) 
# ]
data_run = df_sim_cut_nsbtune
#data_run = df_real_data_stacked

# Define the bins
bins = np.linspace(min_radius_cut, max_radius_cut, 10)
# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)


# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size']
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()


# # Bin the data for MC
# df_real_data_cut_2021['binned_radius'] = pd.cut(df_real_data_cut_2021['ring_radius'], bins, include_lowest=True, labels=False)
# binned_data_2021 = df_real_data_cut_2021.groupby('binned_radius').agg({
#     'ring_radius': 'mean',
#     'ring_size': 'mean'
# })
# binned_data_2021['ring_size'] = binned_data_2021['ring_size']
# binned_data_2021['constant'] = 1
# model_data_2021 = sm.OLS(binned_data_2021['ring_size'], binned_data_2021[['constant', 'ring_radius']]).fit()
# # Generate y values and confidence intervals using the regression models
# y_values_data_2021 = model_data_2021.predict(x_values_with_const)
# prediction_data_2021 = model_data_2021.get_prediction(x_values_with_const)
# conf_int_mc_2021 = prediction_data_2021.conf_int()


# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run['constant'] = 1
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()
y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()


# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
#plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# plt.scatter(binned_data_2021['ring_radius'], binned_data_2021['ring_size'], color='blue', label='Simulation with tuned NSB to Run17043 (0.31 GHz)', s=50)
# plt.plot(x_values, y_values_data_2021, color='blue', linestyle='--', label='MC regression line')
# plt.fill_between(x_values, conf_int_data_2021[:, 0], conf_int_data_2021[:, 1], color='', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label=f'Data for {year_value} year', s=50)

plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
#plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)


# Adding labels and legend
plt.xlabel('Muon radius [deg]',fontsize=16)
plt.ylabel('Muon ring size [p.e.]',fontsize=16)
plt.legend()
plt.ylim(1800,3000)
plt.grid(alpha=0.5)
#plt.title(f"Quality cuts containment > {ring_containment_cut} completeness > {ring_completeness_cut}")
#plt.show()
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/cuts_investigation/size_vs_radius_{year_value}year_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

### RADIUS HIST

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df_sim_cut['ring_radius'],density=True, bins=10, histtype='step', linewidth = 2, label='Simulation')

#plt.hist(df_real_data_cut_2020['ring_radius'],density=True, bins=20, histtype='step', linewidth = 2, label='2020 year')
#plt.hist(df_real_data_cut_2021['ring_radius'],density=True, bins=20, histtype='step', linewidth = 2, label='2021 year')
#plt.hist(df_real_data_cut_2022['ring_radius'],density=True, bins=20, histtype='step', linewidth = 2, label='2022 year')
#plt.hist(df_real_data_cut_2023['ring_radius'],density=True, bins=10, histtype='step', linewidth = 2, label='2023 year')
plt.hist(df_real_data_cut_2024['ring_radius'],density=True, bins=10, histtype='step', linewidth = 2, label='2024 year')

plt.grid(alpha = 0.3)
plt.xlabel('Muon ring radius [deg]')
plt.ylabel('Probability density')
plt.legend(loc = 'upper left', fontsize = 13) 
plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/radius_hist_2024yearVSsim_cont{ring_containment_cut}_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
plt.hist(df_good_data_real['ring_completeness'],density=True, bins = 10, alpha = 0.7, histtype='step', linewidth = 2)
plt.hist(df_sim_zenith10['ring_completeness'],density=True, bins=10, histtype='step', linewidth = 2, label='Simulation')
plt.grid(alpha = 0.3)
plt.xlabel('Ring completeness')
plt.ylabel('Probability density')
#plt.savefig(f'/Users/vdk/muons2024/images/PlanPaper/lst_reco_call/completeness.pdf', dpi=200, format='pdf', bbox_inches='tight')
